In [ ]:
asin2 0.826

Outras similaridades: Cos, Ovl, Jacc, Dice

Semantic Similarity

These models find semantically similar sentences within one language or across languages:

**distiluse-base-multilingual-cased-v2**: Multilingual knowledge distilled version of multilingual Universal Sentence Encoder. While the original mUSE model only supports 16 languages, this multilingual knowledge distilled version supports 50+ languages.

**xlm-r-distilroberta-base-paraphrase-v1** - Multilingual version of distilroberta-base-paraphrase-v1, trained on parallel data for 50+ languages.

**xlm-r-bert-base-nli-stsb-mean-tokens**: Produces similar embeddings as the bert-base-nli-stsb-mean-token model. Trained on parallel data for 50+ languages.

**distilbert-multilingual-nli-stsb-quora-ranking** - Multilingual version of distilbert-base-nli-stsb-quora-ranking. Fine-tuned with parallel data for 50+ languages.

**T-Systems-onsite/cross-en-de-roberta-sentence-transformer** - Multilingual model for English an German. [More]

# Imports e métodos necessários

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

/opt/conda/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
import pandas as pd 
import xml.etree.ElementTree as et 

from scipy.stats import pearsonr

def parse_xml(xml_file):
    """Parse xml to pandas dataframe."""
    xtree = et.parse(xml_file)
    xroot = xtree.getroot() 

    df_cols = ['id', 't', 'h', 'similarity']
    rows = []

    for node in xroot:
        id_ = node.attrib.get("id")
        similarity = node.attrib.get("similarity")
        t = node.find("t").text
        h = node.find("h").text

        rows.append({
            "id": id_,
            "t": t, 
            "h": h,
            "similarity": similarity
        })
    return pd.DataFrame(rows, columns=df_cols, dtype=float)

def eval_similarity(pairs_gold, pairs_sys):
    '''
    Evaluate the semantic similarity output of the system against a gold score. 
    Results are printed to stdout.
    '''
    
    gold_values = np.array(pairs_gold)
    sys_values = np.array(pairs_sys)
    pearson = pearsonr(gold_values, sys_values)[0]
    absolute_diff = gold_values - sys_values
    mse = (absolute_diff ** 2).mean()
    
    print()
    print('Similarity evaluation')
    print('Pearson\t\tMean Squared Error')
    print('-------\t\t------------------')
    print('{:7.3f}\t\t{:18.2f}'.format(pearson, mse))

# Carregando os dados

In [3]:
!ls ../data/assin2

assin2-blind-test.xml  assin2-dev.xml  assin2-test.xml	assin2-train-only.xml


In [4]:
df_ptbr_train = parse_xml('../data/assin2/assin2-train-only.xml')
df_ptbr_dev = parse_xml('../data/assin2/assin2-dev.xml')
df_ptbr_test = parse_xml('../data/assin2/assin2-test.xml')

In [5]:
print(f'assin-ptbr-train: {df_ptbr_train.shape}')
print(f'assin-ptbr-dev: {df_ptbr_dev.shape}')
print(f'assin-ptbr-test: {df_ptbr_test.shape}')

assin-ptbr-train: (6500, 4)
assin-ptbr-dev: (500, 4)
assin-ptbr-test: (2448, 4)


In [6]:
df_ptbr_train.head()

,id,t,h,similarity
0,1.0,Uma criança risonha está segurando uma pistola...,Uma criança está segurando uma pistola de água,4.5
1,2.0,Os homens estão cuidadosamente colocando as ma...,Os homens estão colocando bagagens dentro do p...,4.5
2,3.0,Uma pessoa tem cabelo loiro e esvoaçante e est...,Um guitarrista tem cabelo loiro e esvoaçante,4.7
3,4.0,Batatas estão sendo fatiadas por um homem,O homem está fatiando a batata,4.7
4,5.0,Um caminhão está descendo rapidamente um morro,Um caminhão está rapidamente descendo o morro,4.9


# Testes

## Sentence-BERT

### distiluse-base-multilingual-cased-v2

### xlm-r-distilroberta-base-paraphrase-v1

In [7]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [8]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [9]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [10]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [11]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.794		              0.50


In [12]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.723


### xlm-r-bert-base-nli-stsb-mean-tokens

In [13]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

In [14]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [15]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [16]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [17]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.794		              0.63


In [18]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.760


### distilbert-multilingual-nli-stsb-quora-ranking

In [19]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

In [20]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [21]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [22]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [23]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.720		              1.59


In [24]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.677


### T-Systems-onsite/cross-en-de-roberta-sentence-transformer

In [31]:
from sentence_transformers import models, SentenceTransformer

model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

Exception when trying to download https://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404


In [32]:
t_embeddings = model.encode(df_ptbr_test['t'].tolist())
h_embeddings = model.encode(df_ptbr_test['h'].tolist())

In [33]:
from sklearn.metrics.pairwise import  cosine_similarity

similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

In [34]:
pairs_gold = df_ptbr_test['similarity'].tolist()
pairs_sys = similarities

In [35]:
eval_similarity(pairs_gold, pairs_sys)


Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.804		              0.64


In [36]:
from scipy.stats import spearmanr
print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Spearman correlation:   0.747


## Fine-tuning Sentence-BERT

https://www.sbert.net/docs/training/overview.html

https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/sts/training_stsbenchmark_continue_training.py

### distiluse-base-multilingual-cased-v2

In [7]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer



for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'/mnt/data/sbert_finetuning_assin2_distiluse-base-multilingual-cased-v2-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('distiluse-base-multilingual-cased-v2')

    train_samples = []
    dev_samples = []
    test_samples = []

    for i, row in df_ptbr_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptbr_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    for i, row in df_ptbr_test.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    model = SentenceTransformer(model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
    test_evaluator(model, output_path=model_save_path)

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())

    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]

    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities

    eval_similarity(pairs_gold, pairs_sys)

    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

/opt/conda/lib/python3.7/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.814		              0.49
Spearman correlation:   0.790





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.824		              0.48
Spearman correlation:   0.795





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.819		              0.49
Spearman correlation:   0.788





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.823		              0.48
Spearman correlation:   0.792


...




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.820		              0.50
Spearman correlation:   0.784





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.821		              0.50
Spearman correlation:   0.788





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.820		              0.48
Spearman correlation:   0.791





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.822		              0.48
Spearman correlation:   0.796





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.816		              0.49
Spearman correlation:   0.791


### xlm-r-distilroberta-base-paraphrase-v1

In [8]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'/mnt/data/sbert_finetuning_assin2_xlm-r-distilroberta-base-paraphrase-v1-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('xlm-r-distilroberta-base-paraphrase-v1')

    train_samples = []
    dev_samples = []
    test_samples = []

    for i, row in df_ptbr_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptbr_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    for i, row in df_ptbr_test.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    model = SentenceTransformer(model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
    test_evaluator(model, output_path=model_save_path)
    
    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())
    
    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    
    eval_similarity(pairs_gold, pairs_sys)
    
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.850		              0.42
Spearman correlation:   0.819





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.850		              0.42
Spearman correlation:   0.817





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.848		              0.43
Spearman correlation:   0.815





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.848		              0.44
Spearman correlation:   0.815





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.848		              0.43
Spearman correlation:   0.816





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.851		              0.43
Spearman correlation:   0.818





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.848		              0.43
Spearman correlation:   0.816





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.851		              0.42
Spearman correlation:   0.818





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.848		              0.43
Spearman correlation:   0.816





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.849		              0.43
Spearman correlation:   0.818


### xlm-r-bert-base-nli-stsb-mean-tokens

In [9]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'/mnt/data/sbert_finetuning_assin2_xlm-r-bert-base-nli-stsb-mean-tokens-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('xlm-r-bert-base-nli-stsb-mean-tokens')

    train_samples = []
    dev_samples = []
    test_samples = []

    for i, row in df_ptbr_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptbr_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    for i, row in df_ptbr_test.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    model = SentenceTransformer(model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
    test_evaluator(model, output_path=model_save_path)

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())
    
    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    
    eval_similarity(pairs_gold, pairs_sys)
    
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.836		              0.43
Spearman correlation:   0.810





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.837		              0.42
Spearman correlation:   0.811





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.843		              0.42
Spearman correlation:   0.814





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.837		              0.42
Spearman correlation:   0.809





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.839		              0.43
Spearman correlation:   0.810





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.840		              0.42
Spearman correlation:   0.812





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.838		              0.42
Spearman correlation:   0.808





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.835		              0.43
Spearman correlation:   0.810





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.841		              0.42
Spearman correlation:   0.814





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.841		              0.42
Spearman correlation:   0.813


### distilbert-multilingual-nli-stsb-quora-ranking

In [10]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'/mnt/data/sbert_finetuning_assin2_distilbert-multilingual-nli-stsb-quora-ranking-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('distilbert-multilingual-nli-stsb-quora-ranking')

    train_samples = []
    dev_samples = []
    test_samples = []

    for i, row in df_ptbr_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptbr_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    for i, row in df_ptbr_test.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    model = SentenceTransformer(model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
    test_evaluator(model, output_path=model_save_path)

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())
    
    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    
    eval_similarity(pairs_gold, pairs_sys)
    
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.829		              0.47
Spearman correlation:   0.801





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.827		              0.47
Spearman correlation:   0.797





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.829		              0.48
Spearman correlation:   0.797





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.828		              0.48
Spearman correlation:   0.799





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.832		              0.47
Spearman correlation:   0.804





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.833		              0.46
Spearman correlation:   0.806





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.830		              0.47
Spearman correlation:   0.798





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.828		              0.47
Spearman correlation:   0.800





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.833		              0.46
Spearman correlation:   0.805





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.831		              0.46
Spearman correlation:   0.802


### T-Systems-onsite/cross-en-de-roberta-sentence-transformer

In [11]:
"""
This example loads the pre-trained SentenceTransformer model 'bert-base-nli-mean-tokens' from the server.
It then fine-tunes this model for some epochs on the STS benchmark dataset.
Note: In this example, you must specify a SentenceTransformer model.
If you want to fine-tune a huggingface/transformers model like bert-base-uncased, see training_nli.py and training_stsbenchmark.py
"""
from torch.utils.data import DataLoader
import math
from sentence_transformers import SentenceTransformer,  SentencesDataset, LoggingHandler, losses, util, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from datetime import datetime
import os
import csv
from sentence_transformers import models, SentenceTransformer


for i in range(1, 10 + 1):
    # Read the dataset
    train_batch_size = 16
    num_epochs = 4
    model_save_path = f'/mnt/data/sbert_finetuning_assin2_T-Systems-onsite_cross-en-de-roberta-sentence-transformer-{i}'# + datetime.now().strftime("%Y-%m-%d_%H-%M-%S")



    # Load a pre-trained sentence transformer model
    #model = SentenceTransformer(model_name)


    model = SentenceTransformer('T-Systems-onsite/cross-en-de-roberta-sentence-transformer')

    train_samples = []
    dev_samples = []
    test_samples = []

    for i, row in df_ptbr_train.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        train_samples.append(inp_example)

    for i, row in df_ptbr_dev.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        dev_samples.append(inp_example)

    for i, row in df_ptbr_test.iterrows():
        inp_example = InputExample(texts=[row['t'], row['h']], label=row['similarity'] / 5)
        test_samples.append(inp_example)



    train_dataset = SentencesDataset(train_samples, model)
    train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=train_batch_size)
    train_loss = losses.CosineSimilarityLoss(model=model)


    evaluator = EmbeddingSimilarityEvaluator.from_input_examples(dev_samples, name='sts-dev')


    # Configure the training. We skip evaluation in this example
    warmup_steps = math.ceil(len(train_dataset) * num_epochs / train_batch_size * 0.1) #10% of train data for warm-up


    # Train the model
    model.fit(train_objectives=[(train_dataloader, train_loss)],
              evaluator=evaluator,
              epochs=num_epochs,
              evaluation_steps=1000,
              warmup_steps=warmup_steps,
              output_path=model_save_path)


    ##############################################################################
    #
    # Load the stored model and evaluate its performance on STS benchmark dataset
    #
    ##############################################################################

    model = SentenceTransformer(model_save_path)
    test_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(test_samples, name='sts-test')
    test_evaluator(model, output_path=model_save_path)

    t_embeddings = model.encode(df_ptbr_test['t'].tolist())
    h_embeddings = model.encode(df_ptbr_test['h'].tolist())
    
    from sklearn.metrics.pairwise import  cosine_similarity

    similarities = [5.0 * cosine_similarity([t], [h])[0][0] for t, h in zip(t_embeddings, h_embeddings)]
    
    pairs_gold = df_ptbr_test['similarity'].tolist()
    pairs_sys = similarities
    
    eval_similarity(pairs_gold, pairs_sys)
    
    from scipy.stats import spearmanr
    print('Spearman correlation: {:7.3f}'.format(spearmanr(pairs_gold, pairs_sys)[0]))

Exception when trying to download https://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.848		              0.41
Spearman correlation:   0.814


Exception when trying to download https://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.851		              0.41
Spearman correlation:   0.818


Exception when trying to download https://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404


...




Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.848		              0.42
Spearman correlation:   0.819


Exception when trying to download https://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.853		              0.41
Spearman correlation:   0.820


Exception when trying to download https://sbert.net/models/T-Systems-onsite/cross-en-de-roberta-sentence-transformer.zip. Response 404





Similarity evaluation
Pearson		Mean Squared Error
-------		------------------
  0.853		              0.40
Spearman correlation:   0.819
